In [11]:
import pandas as pd
from functions import predict, pipe, license_plate,check,thresh
from tqdm import tqdm

In [12]:

df=pd.read_csv('/Users/nour/Documents/OKP4/detection-of-personal-data/data_test/personal_data_db_translated.csv')
df=df.dropna()
df

,Unnamed: 0,sentence,label
0,0,"My name is Laval Jacquin, I'm 38 years old, my...",1.0
1,1,Nour Kired and Damien Sonneville are good alte...,1.0
2,2,Arnaud Mimart is the head of the chocoPolice,1.0
3,3,yeah that's what i i just did today i got uh D...,0.0
4,4,"Founded in 1861, the Massachusetts Institute o...",0.0
...,...,...,...
409,409,"In 1972, Phillip Morris, Inc.'s Miller Brewing...",0.0
410,410,Here you will find plenty of copies of all loc...,0.0
411,411,Although the members of the CVR committee cons...,0.0
412,412,The company's ISD works with ISD officials or ...,0.0


In [10]:
labels=pii_detect(df)
labels

  3%|▎         | 14/414 [14:24<6:51:26, 61.72s/it]  


KeyboardInterrupt: 

In [5]:
df[:3]

,Unnamed: 0,sentence,label
0,0,"My name is Laval Jacquin, I'm 38 years old, my...",1.0
1,1,Nour Kired and Damien Sonneville are good alte...,1.0
2,2,Arnaud Mimart is the head of the chocoPolice,1.0


In [13]:
# candidate_labels = ['First and last name', 'Mail','Internet protocol (IP) addresses', 'location', 'Date of birth', 'Date of death', 'Place of birth', 'Place of death', 'IP', 'a cookie', 'phone', 'health data','private data','personal data','not personal data','not private data']
name=["person's name","person"]
birth=["birthday","private"]
# ip=["Internet protocol (IP) addresses","IP adress"]
cookie=["cookie identifiers","a cookie"]
phone=["phone numbers","phone number","telphone number"]
mail=["mail addresses","mail address","mail"]
location=["location adress","private postal address"]
health=["health data","medical records","health information","medical information","Medical Procedure names","Medical Procedure identification codes"]
personal=["personal data","personal","personal information","private data","private information"]
not_personal=["not personal data","not personal information","not personal","not private data","not private","not private information"]
passport=['passport','code']
Driving_license=['driving license','code']
social_security_number=['social security number']
Tax_file_number=['tax file number']
credit_card=['credit card number']
candidate_labels=name+cookie+phone+mail+location+health+personal+not_personal+birth+passport+Driving_license+social_security_number+Tax_file_number+credit_card


In [14]:
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number"),(credit_card,"credit_card")]

In [27]:

def predict(sentence : str,treshold : float):
    result_details=pipe(sentence, candidate_labels, multi_label=True)
    result_details=dict(zip(result_details['labels'],result_details['scores']))
    result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
    pii_detected={name : result[name] for name in thresh(result,treshold)}
    pii_detected=check(pii_detected,sentence)
    if license_plate(sentence):
            pii_detected["license_plate"]=0.9
    if pii_detected=={}:
            mean=0
    else:
            mean=np.mean(list(pii_detected.values()))
    del result_details,result
    return int(pii_detected!={}),mean



In [20]:
df

,Unnamed: 0,sentence,label
0,0,"My name is Laval Jacquin, I'm 38 years old, my...",1.0
1,1,Nour Kired and Damien Sonneville are good alte...,1.0
2,2,Arnaud Mimart is the head of the chocoPolice,1.0
3,3,yeah that's what i i just did today i got uh D...,0.0
4,4,"Founded in 1861, the Massachusetts Institute o...",0.0
...,...,...,...
409,409,"In 1972, Phillip Morris, Inc.'s Miller Brewing...",0.0
410,410,Here you will find plenty of copies of all loc...,0.0
411,411,Although the members of the CVR committee cons...,0.0
412,412,The company's ISD works with ISD officials or ...,0.0


In [25]:
from tqdm import tqdm
import numpy as np

In [28]:
y_pred=[]
predicted_proba=[]
for sent in tqdm(df['sentence']):
    result=predict(sent,0.9)
    y_pred.append(result[0])
    predicted_proba.append(result[1])

  1%|▏         | 6/414 [03:20<3:47:13, 33.42s/it]


KeyboardInterrupt: 

In [19]:
df['sentence']

0      My name is Laval Jacquin, I'm 38 years old, my...
1      Nour Kired and Damien Sonneville are good alte...
2           Arnaud Mimart is the head of the chocoPolice
3      yeah that's what i i just did today i got uh D...
4      Founded in 1861, the Massachusetts Institute o...
                             ...                        
409    In 1972, Phillip Morris, Inc.'s Miller Brewing...
410    Here you will find plenty of copies of all loc...
411    Although the members of the CVR committee cons...
412    The company's ISD works with ISD officials or ...
413    You can buy several of these to make your own ...
Name: sentence, Length: 414, dtype: object

In [7]:
conf = confusion_matrix(df['label'], y_pred)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,289,103
vrai 1,8,14


In [11]:
np.mean([df.label==y_pred])

0.7318840579710145

In [9]:
result=df.copy()
result['predicted']=y_pred
result_no_match=result[result.label!=y_pred]
result_no_match.to_csv('./data_test/result_no_match.csv')
result_no_match

,sentence,label,predicted
8,"Je suis euh, grand sergent chef, à la retraite...",0,1
9,The FBI legal attach�'s office in Paris first ...,0,1
15,"In Prinsengracht, Otto Frank and his family hi...",0,1
22,He joined the rest of his team at their hotel.,0,1
24,"Je mange réellement vite, aussi vite que je pe...",0,1
...,...,...,...
392,Davidson should not adopt the pronunciation of...,0,1
394,"Et je pensais que c'était un privilège, et ça ...",0,1
403,You may have heard of me. Captain Calverley ...,0,1
404,"Pour plus de détails, voir //www.healtheffects...",0,1


In [5]:
result_no_match=pd.read_csv('./data_test/result_no_match.csv')
y_pred_06=[]
predicted_proba=[]
for sent in tqdm(result_no_match['sentence']):
    result=predict(sent,treshold=0.6)
    y_pred_06.append(result[0])
    predicted_proba.append(result[1])

100%|██████████| 111/111 [16:14<00:00,  8.78s/it]


0.44144144144144143

In [8]:
conf = confusion_matrix(result_no_match['label'], y_pred_06)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,49,54
vrai 1,8,0


In [11]:
result_no_match=pd.read_csv('./data_test/result_no_match.csv')
y_pred_06=[]
predicted_proba=[]
for sent in tqdm(result_no_match['sentence']):
    result=predict(sent,treshold=0.7)
    y_pred_06.append(result[0])
    predicted_proba.append(result[1])

np.mean([result_no_match.label==y_pred_06])

100%|██████████| 111/111 [15:27<00:00,  8.36s/it]


0.6216216216216216

In [12]:
conf = confusion_matrix(result_no_match['label'], y_pred_06)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,69,34
vrai 1,8,0


In [13]:
result_no_match=pd.read_csv('./data_test/result_no_match.csv')
y_pred_08=[]
predicted_proba=[]
for sent in tqdm(result_no_match['sentence']):
    result=predict(sent,treshold=0.8)
    y_pred_08.append(result[0])
    predicted_proba.append(result[1])

np.mean([result_no_match.label==y_pred_08])


100%|██████████| 111/111 [17:10<00:00,  9.28s/it]


0.6666666666666666

In [14]:
conf = confusion_matrix(result_no_match['label'], y_pred_08)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,74,29
vrai 1,8,0


In [16]:
result_no_match=pd.read_csv('./data_test/result_no_match.csv')
y_pred_75=[]
predicted_proba=[]
for sent in tqdm(result_no_match['sentence']):
    result=predict(sent,treshold=0.9)
    y_pred_75.append(result[0])
    predicted_proba.append(result[1])

np.mean([result_no_match.label==y_pred_75])


100%|██████████| 111/111 [11:43<00:00,  6.34s/it]


0.7927927927927928

In [17]:
conf = confusion_matrix(result_no_match['label'], y_pred_75)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,88,15
vrai 1,8,0
